# Reviews Text Sentiment Analysis For Thailand Tourism Recommendation

This project target for create recommendation system from rating thailand tourism by learning review positive or negative

#### Step #1 - Setup Project Library Package

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
import os
from geopy.geocoders import Nominatim
from dotenv import load_dotenv

load_dotenv()

True

#### Step #2 - Data Preparation
##### 2.1 Thailand Province -> Amphure Data <br>
https://github.com/kongvut/thai-province-data

In [3]:
province_data = pd.read_json('hook/data/api_province_with_amphure_tambon.json')
province_data

,id,name_th,name_en,geography_id,created_at,updated_at,deleted_at,amphure
0,1,กรุงเทพมหานคร,Bangkok,2,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 1001, 'name_th': 'เขตพระนคร', 'name_en..."
1,2,สมุทรปราการ,Samut Prakan,2,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 1101, 'name_th': 'เมืองสมุทรปราการ', '..."
2,3,นนทบุรี,Nonthaburi,2,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 1201, 'name_th': 'เมืองนนทบุรี', 'name..."
3,4,ปทุมธานี,Pathum Thani,2,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 1301, 'name_th': 'เมืองปทุมธานี', 'nam..."
4,5,พระนครศรีอยุธยา,Phra Nakhon Si Ayutthaya,2,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 1401, 'name_th': 'พระนครศรีอยุธยา', 'n..."
...,...,...,...,...,...,...,...,...
72,73,พัทลุง,Phatthalung,6,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 9301, 'name_th': 'เมืองพัทลุง', 'name_..."
73,74,ปัตตานี,Pattani,6,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 9401, 'name_th': 'เมืองปัตตานี', 'name..."
74,75,ยะลา,Yala,6,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 9501, 'name_th': 'เมืองยะลา', 'name_en..."
75,76,นราธิวาส,Narathiwat,6,2019-08-09 03:33:09+07:00,2022-05-16 06:31:03+07:00,NaT,"[{'id': 9601, 'name_th': 'เมืองนราธิวาส', 'nam..."


In [4]:
provinces = province_data['name_en'].tolist()
geolocator = Nominatim(user_agent="thailand_provinces")
data = {'Province': [], 'Latitude': [], 'Longitude': []}

for province in provinces:
    location = geolocator.geocode(f"{province}, Thailand")
    if location:
        data['Province'].append(province)
        data['Latitude'].append(location.latitude)
        data['Longitude'].append(location.longitude)

thailland_province_coordinates = pd.DataFrame(data)


In [4]:
# thailland_province_coordinates.to_csv("thailand_province_coordinate.csv", index=False, header=True, sep=',', encoding='utf-8')

In [5]:
thailland_province_coordinates[thailland_province_coordinates['Province'] == 'Bangkok']

,Province,Latitude,Longitude
0,Bangkok,13.752494,100.493509


##### 2.2 Thailand Tourism Location API Data <br>
https://developers.tourismthailand.org/

In [5]:
import hook.thai_tourism_data as tat
tourism_location = tat.ThaiTourismAPI("TH")

def getTourismLocationsByProvince(province_name, province_location, category):
    search_params = {
            "keyword": "",  # Example keyword for Kanchanaburi
            "location": province_location,
            "categorycodes": category,
            "province_name": province_name,
            "radius": 1000,
            "number_of_result": 100,
            "page_number": 1,
            "destination": province_name,
            "filter_by_update_date": "2019/09/01-2024/05/31"
        }
    tourism_data = tourism_location.fetch_search_location(**search_params)

    df = pd.DataFrame([vars(place) for place in tourism_data])
    return df


In [8]:
tourism_attraction_locations = pd.DataFrame()
tourism_hotels_locations = pd.DataFrame()
tourism_restaurants_locations = pd.DataFrame()
tourism_shops_locations = pd.DataFrame()
# CATEGORIES: ATTRACTION | ACCOMMODATION  | RESTAURANT | GEOS | SHOP

for index, pvn in province_data.iterrows():
    coordinate = thailland_province_coordinates[thailland_province_coordinates['Province'] == pvn['name_en']]
    
    if len(coordinate['Latitude'].values) > 0:
        tourism_shops_location = getTourismLocationsByProvince(pvn['name_th'], 
        f"{coordinate['Latitude'].values[0]},"
        f"{coordinate['Longitude'].values[0]}",
        "SHOP")
        
        tourism_shops_locations = pd.concat([tourism_shops_locations, tourism_shops_location], ignore_index=True)
        
        province_torism_location = getTourismLocationsByProvince(pvn['name_th'], 
        f"{coordinate['Latitude'].values[0]},"
        f"{coordinate['Longitude'].values[0]}",
        "ATTRACTION")
        
        tourism_attraction_locations = pd.concat([tourism_attraction_locations, province_torism_location], ignore_index=True)
        
        tourism_hotels_location = getTourismLocationsByProvince(pvn['name_en'], 
        f"{coordinate['Latitude'].values[0]},"
        f"{coordinate['Longitude'].values[0]}",
        "ACCOMMODATION")
        
        tourism_hotels_locations = pd.concat([tourism_hotels_locations, tourism_hotels_location], ignore_index=True)
        
        tourism_restaurants_location = getTourismLocationsByProvince(pvn['name_th'], 
        f"{coordinate['Latitude'].values[0]},"
        f"{coordinate['Longitude'].values[0]}",
        "RESTAURANT")
        
        tourism_restaurants_locations = pd.concat([tourism_restaurants_locations, tourism_restaurants_location], ignore_index=True)
        

HTTP Request failed:  404 Client Error: Not Found for url: https://tatapi.tourismthailand.org/tatapi/v5/places/search?keyword=&location=14.9936716%2C100.3602963&categorycodes=ACCOMMODATION&provinceName=Sing+Buri&radius=1000&numberOfResult=100&pagenumber=1
HTTP Request failed:  404 Client Error: Not Found for url: https://tatapi.tourismthailand.org/tatapi/v5/places/search?keyword=&location=16.2720349%2C101.7153824&categorycodes=ACCOMMODATION&provinceName=Chaiyaphum&radius=1000&numberOfResult=100&pagenumber=1
HTTP Request failed:  404 Client Error: Not Found for url: https://tatapi.tourismthailand.org/tatapi/v5/places/search?keyword=&location=15.9949473%2C104.7328575&categorycodes=ACCOMMODATION&provinceName=Amnat+Charoen&radius=1000&numberOfResult=100&pagenumber=1
HTTP Request failed:  404 Client Error: Not Found for url: https://tatapi.tourismthailand.org/tatapi/v5/places/search?keyword=&location=15.7486448%2C103.7013017&categorycodes=ACCOMMODATION&provinceName=Roi+Et&radius=1000&number

In [9]:
tourism_shops_locations

,place_id,place_name,latitude,longitude,category_code,category_description,sha_name,sha_type_code,sha_type_description,sha_cate_id,...,address,sub_district,district,province,postcode,thumbnail_url,destination,tags,distance,update_date
0,P06000698,32 มอเตอร์ไซค์,13.745079,100.470073,SHOP,ร้านค้า,,,,,...,,บางขุนศรี,บางกอกน้อย,กรุงเทพมหานคร,10700,,กรุงเทพมหานคร,None,0.0,2023-10-01T17:45:34.463
1,P06000411,9 เค การช่าง,13.607451,100.426857,SHOP,ร้านค้า,,,,,...,17/673,แสมดำ,บางขุนเทียน,กรุงเทพมหานคร,10150,,กรุงเทพมหานคร,None,0.0,2019-12-13T12:39:06.313
2,P06002110,JIM THOMPSON,13.730429,100.533504,SHOP,ร้านค้า,,,,,...,9 ถนนสุรวงศ์,สุริยวงศ์,บางรัก,กรุงเทพมหานคร,10500,https://tatapi.tourismthailand.org/tatfs/Image...,กรุงเทพมหานคร,None,0.0,2021-12-29T13:47:26.167
3,P06002136,JIM THOMPSON,13.730429,100.533504,SHOP,ร้านค้า,,,,,...,9 ถนนสุรวงศ์,สุริยวงศ์,บางรัก,กรุงเทพมหานคร,10500,https://tatapi.tourismthailand.org/tatfs/IMAGE...,BANGKOK,None,0.0,2020-08-31T16:14:34
4,P06000004,กลุ่มดอกไม้ประดิษฐ์สินเกล้า,13.748388,100.740892,SHOP,ร้านค้า,,,,,...,198/242 หมู่ 5 ถนนร่มเกล้า,คลองสามประเวศ,ลาดกระบัง,กรุงเทพมหานคร,10520,,กรุงเทพมหานคร,None,0.0,2019-11-12T10:52:22.553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2071,P06000830,โอทอปองค์การบริหารส่วนจังหวัดพัทลุง,7.634606,100.041973,SHOP,ร้านค้า,,,,,...,,เขาเจียก,เมืองพัทลุง,พัทลุง,93000,,พัทลุง,None,0.0,2023-10-03T21:19:54.84
2072,P06001472,อาซัน,6.860527,101.255057,SHOP,ร้านค้า,,,,,...,,จะบังติกอ,เมืองปัตตานี,ปัตตานี,94000,,ปัตตานี,None,0.0,2023-10-07T12:12:15.303
2073,P06000203,โยฮันบาติก,6.020624,101.957939,SHOP,ร้านค้า,,,,,...,,สุไหงโก-ลก,สุไหงโก-ลก,นราธิวาส,96120,https://tatapi.tourismthailand.org/tatfs/Image...,นราธิวาส,None,0.0,2020-01-16T17:14:09.133
2074,P06000937,ศูนย์จำหน่ายสินค้าโอทอปอำเภอโซ่พิสัย,18.080719,103.440438,SHOP,ร้านค้า,,,,,...,,โซ่,โซ่พิสัย,บึงกาฬ,38170,,บึงกาฬ,None,0.0,2023-10-03T21:38:13.28


In [8]:
tourism_attraction_locations

,place_id,place_name,latitude,longitude,category_code,category_description,sha_name,sha_type_code,sha_type_description,sha_cate_id,...,address,sub_district,district,province,postcode,thumbnail_url,destination,tags,distance,update_date
0,P03002579,พระบรมมหาราชวัง,13.749831,100.491289,ATTRACTION,สถานที่ท่องเที่ยว,พระบรมมหาราชวัง,3,นันทนาการ และสถานที่ท่องเที่ยว,1,...,,พระบรมมหาราชวัง,พระนคร,กรุงเทพมหานคร,10200,https://tatapi.tourismthailand.org/tatfs/Image...,กรุงเทพมหานคร,None,0.0,2024-02-12T10:22:28.337
1,P03006845,พระปรางค์วัดอรุณราชวรารามราชวรมหาวิหาร,13.743797,100.488963,ATTRACTION,สถานที่ท่องเที่ยว,,,,,...,,วัดอรุณ,บางกอกใหญ่,กรุงเทพมหานคร,10600,https://tatapi.tourismthailand.org/tatfs/Image...,กรุงเทพมหานคร,None,0.0,2024-02-07T14:29:29.31
2,P03007639,มิวเซียมสยาม พิพิธภัณฑ์การเรียนรู้,13.744175,100.494165,ATTRACTION,สถานที่ท่องเที่ยว,มิวเซียมสยาม,3,นันทนาการ และสถานที่ท่องเที่ยว,2,...,ถนนสนามไชย,พระบรมมหาราชวัง,พระนคร,กรุงเทพมหานคร,10200,https://tatapi.tourismthailand.org/tatfs/Image...,กรุงเทพมหานคร,None,0.0,2024-02-23T15:17:07.253
3,P03012239,ศาลท้าวมหาพรหม,13.744251,100.540397,ATTRACTION,สถานที่ท่องเที่ยว,,,,,...,,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,10330,https://tatapi.tourismthailand.org/tatfs/Image...,กรุงเทพมหานคร,None,0.0,2024-02-12T14:03:05.2
4,P03007495,เสาชิงช้า,13.751819,100.501298,ATTRACTION,สถานที่ท่องเที่ยว,,,,,...,,เสาชิงช้า,พระนคร,กรุงเทพมหานคร,10200,https://tatapi.tourismthailand.org/tatfs/Image...,กรุงเทพมหานคร,None,0.0,2024-02-12T13:52:05.74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4169,P03012246,หนองกุดทิงใหญ่,18.296982,103.662169,ATTRACTION,สถานที่ท่องเที่ยว,,,,,...,,โนนสมบูรณ์,เมืองบึงกาฬ,บึงกาฬ,38000,https://tatapi.tourismthailand.org/tatfs/IMAGE...,บึงกาฬ,None,0.0,2020-02-13T08:00:00
4170,P03018256,หาดคำสมบูรณ์,17.983004,104.028832,ATTRACTION,สถานที่ท่องเที่ยว,,,,,...,,บึงโขงหลง,บึงโขงหลง,บึงกาฬ,38220,,บึงกาฬ,None,0.0,2023-12-19T13:52:46.38
4171,P03012249,หาดทรายขาวริมโขง,18.372496,103.649679,ATTRACTION,สถานที่ท่องเที่ยว,,,,,...,,วิศิษฐ์,เมืองบึงกาฬ,บึงกาฬ,38000,https://tatapi.tourismthailand.org/tatfs/IMAGE...,บึงกาฬ,None,0.0,2023-10-16T12:47:02.647
4172,P03004782,หาดสีดา,18.329346,103.973340,ATTRACTION,สถานที่ท่องเที่ยว,,,,,...,บ้านหนองเดิ่นท่า ตำบลหนองเดิ่น,หนองเดิ่น,บุ่งคล้า,บึงกาฬ,38000,,บึงกาฬ,None,0.0,2020-08-28T14:49:07.987


In [9]:
tourism_hotels_locations

,place_id,place_name,latitude,longitude,category_code,category_description,sha_name,sha_type_code,sha_type_description,sha_cate_id,...,address,sub_district,district,province,postcode,thumbnail_url,destination,tags,distance,update_date
0,P02000010,เซ็นทารา แกรนด์ แอท เซ็นทรัลพลาซา ลาดพร้าว กรุ...,13.818145,100.559918,ACCOMMODATION,โรงแรม,โรงแรมเซ็นทาราแกรนด์ เซ็นทรัลพลาซา ลาดพร้าว กร...,2,โรงแรมและที่พัก สถานที่จัดการประชุม,3,...,1695 ถนนพหลโยธิน,จตุจักร,จตุจักร,กรุงเทพมหานคร,10900,https://tatapi.tourismthailand.org/tatfs/Image...,กรุงเทพมหานคร,None,0.0,2019-09-11T13:44:08.187
1,P02000087,โซ โซฟิเทล แบงค็อก,13.726319,100.543080,ACCOMMODATION,โรงแรม,,,,,...,188 ถนนสีลม,สีลม,บางรัก,กรุงเทพมหานคร,10500,https://tatapi.tourismthailand.org/tatfs/Image...,กรุงเทพมหานคร,None,0.0,2019-09-11T15:21:13.793
2,P02012321,โรงแรม ดิ โอกุระ เพรสทีจ กรุงเทพฯ,13.742755,100.548103,ACCOMMODATION,โรงแรม,โรงแรม ดิ โอกุระ เพรสทีจ กรุงเทพฯ,2,โรงแรมและที่พัก สถานที่จัดการประชุม,3,...,57 ถนนวิทยุ,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,10330,https://tatapi.tourismthailand.org/tatfs/Image...,กรุงเทพมหานคร,None,0.0,2019-09-11T17:19:49.387
3,P02000168,โรงแรมแกรนด์ เซ็นเตอร์พอยต์ ราชดำริ,13.741951,100.541125,ACCOMMODATION,โรงแรม,โรงแรมแกรนด์ เซนเตอร์พอยต์ ราชดำริ,2,โรงแรมและที่พัก สถานที่จัดการประชุม,3,...,153/2 ซอยมหาดเล็กหลวง 1 ถนนราชดำริ,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,10330,https://tatapi.tourismthailand.org/tatfs/Image...,กรุงเทพมหานคร,None,0.0,2019-09-11T16:29:26.46
4,P02000016,โรงแรมโกลเด้น ทิวลิป ซอฟเฟอริน กรุงเทพ,13.753446,100.572209,ACCOMMODATION,โรงแรม,โรงแรมโกลเด้น ทิวลิป ซอฟเฟอริน กรุงเทพ,2,โรงแรมและที่พัก สถานที่จัดการประชุม,3,...,92 ซอยแสงจันทร์ ถนนพระราม 9,บางกะปิ,ห้วยขวาง,กรุงเทพมหานคร,10310,https://tatapi.tourismthailand.org/tatfs/Image...,กรุงเทพมหานคร,None,0.0,2019-09-11T13:58:11.593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1377,P02015591,โอโชนรีสอร์ทแอนด์พูลวิลล่า,7.475757,100.076673,ACCOMMODATION,โรงแรม,โอโชนรีสอร์ทแอนด์พูลวิลล่า,2,โรงแรมและที่พัก สถานที่จัดการประชุม,1,...,413 หมู่ 1 บ้านนาหยา,ควนขนุน,เขาชัยสน,พัทลุง,93130,https://tatapi.tourismthailand.org/tatfs/Image...,พัทลุง,None,0.0,2023-10-16T18:00:57.403
1378,P02017687,โฮมสเตย์กระจูดวรรณี,7.791026,100.097394,ACCOMMODATION,โรงแรม,โฮมสเตย์กระจูดวรรณี,2,โรงแรมและที่พัก สถานที่จัดการประชุม,1,...,152 หมู่ 10,พนางตุง,ควนขนุน,พัทลุง,93110,,พัทลุง,None,0.0,2023-10-16T17:59:27.403
1379,P02000139,โรงแรมซีเอส ปัตตานี,6.863997,101.233701,ACCOMMODATION,โรงแรม,โรงแรม ซี.เอส.ปัตตานี,2,โรงแรมและที่พัก สถานที่จัดการประชุม,2,...,299 หมู่ 4 ถนนหนองจิก,รูสะมิแล,เมืองปัตตานี,ปัตตานี,94000,https://tatapi.tourismthailand.org/tatfs/Image...,ปัตตานี,None,0.0,2019-09-11T16:05:11.4
1380,P02014059,โรงแรมคาเธ่ย์เบตง,5.772894,101.069917,ACCOMMODATION,โรงแรม,โรงแรมคาเธ่ย์เบตง,2,โรงแรมและที่พัก สถานที่จัดการประชุม,2,...,18 ถนนจันทโรทัย,เบตง,เบตง,ยะลา,95110,https://tatapi.tourismthailand.org/tatfs/Image...,ยะลา,None,0.0,2021-03-10T18:26:23.7


In [10]:
tourism_restaurants_locations

,place_id,place_name,latitude,longitude,category_code,category_description,sha_name,sha_type_code,sha_type_description,sha_cate_id,...,address,sub_district,district,province,postcode,thumbnail_url,destination,tags,distance,update_date
0,P08000887,กินซ่าซูชิอิชิ,13.743490,100.540573,RESTAURANT,ร้านอาหาร กาแฟ เบเกอรี่,,,,,...,,ลุมพินี,ปทุมวัน,กรุงเทพมหานคร,10330,https://tatapi.tourismthailand.org/tatfs/Image...,กรุงเทพมหานคร,None,0.0,2019-09-24T09:04:11.437
1,P08000644,ฉั่วคิมเฮง,13.740378,100.602109,RESTAURANT,ร้านอาหาร กาแฟ เบเกอรี่,,,,,...,"81,83 ถนนพัฒนาการ",สวนหลวง,สวนหลวง,กรุงเทพมหานคร,10110,https://tatapi.tourismthailand.org/tatfs/Image...,กรุงเทพมหานคร,None,0.0,2019-09-12T17:58:55.443
2,P08020594,@ซอยเก้า,13.677749,100.651555,RESTAURANT,ร้านอาหาร กาแฟ เบเกอรี่,@ซอยเก้า,1,ภัตตาคาร/ร้านอาหาร,1,...,60/21 ถนนเฉลิมพระเกียรติ ร.9,หนองบอน,ประเวศ,กรุงเทพมหานคร,10260,,กรุงเทพมหานคร,None,0.0,2023-10-22T15:31:39.023
3,P08012291,100 มหาเศรษฐ์,13.728986,100.518298,RESTAURANT,ร้านอาหาร กาแฟ เบเกอรี่,100 มหาเศรษฐ์ สี่พระยา - เอกมัย,1,ภัตตาคาร/ร้านอาหาร,2,...,104 ถนนมหาเศรษฐ์ แขวงสี่พระยา เขตบางรัก กรุงเท...,สี่พระยา,บางรัก,กรุงเทพมหานคร,10500,https://tatapi.tourismthailand.org/tatfs/Image...,กรุงเทพมหานคร,None,0.0,2022-03-11T18:30:00
4,P08021103,101 อาหาร+ดื่ม,13.691278,100.614209,RESTAURANT,ร้านอาหาร กาแฟ เบเกอรี่,101 อาหาร+ดื่ม,1,ภัตตาคาร/ร้านอาหาร,1,...,694 สุขุมวิท 101,บางจาก,พระโขนง,กรุงเทพมหานคร,10260,,กรุงเทพมหานคร,None,0.0,2023-10-22T15:00:40.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3774,P08012251,ครัวบ้านป่า,18.025617,103.706855,RESTAURANT,ร้านอาหาร กาแฟ เบเกอรี่,,,,,...,,พรเจริญ,พรเจริญ,บึงกาฬ,38180,https://tatapi.tourismthailand.org/tatfs/IMAGE...,บึงกาฬ,None,0.0,2019-12-12T17:21:36.24
3775,P08001011,ครัวผู้ว่า 2000,18.367200,103.653345,RESTAURANT,ร้านอาหาร กาแฟ เบเกอรี่,ครัวผู้ว่า 2000,1,ภัตตาคาร/ร้านอาหาร,1,...,ถนนประสาทชัย,บึงกาฬ,เมืองบึงกาฬ,บึงกาฬ,38000,https://tatapi.tourismthailand.org/tatfs/Image...,บึงกาฬ,None,0.0,2019-10-04T13:35:16.35
3776,P08001010,ครัวเสวย,18.371309,103.650084,RESTAURANT,ร้านอาหาร กาแฟ เบเกอรี่,,,,,...,ถนนประสาทชัย,วิศิษฐ์,เมืองบึงกาฬ,บึงกาฬ,38000,,บึงกาฬ,None,0.0,2019-10-04T13:34:50.933
3777,P08001009,มาวินโอชา,18.366506,103.652558,RESTAURANT,ร้านอาหาร กาแฟ เบเกอรี่,,,,,...,ถนนประสาทชัย,บึงกาฬ,เมืองบึงกาฬ,บึงกาฬ,38000,,บึงกาฬ,None,0.0,2019-10-04T13:34:26.247


In [10]:
tourism_shops_locations.to_csv("tourism_shops_locations.csv", index=False, na_rep='N/A', header=True, sep=',', encoding='utf-8')

tourism_attraction_locations.to_csv("tourism_attraction_locations.csv", index=False, na_rep='N/A', header=True, sep=',', encoding='utf-8')

tourism_hotels_locations.to_csv("tourism_hotels_locations.csv", index=False, na_rep='N/A', header=True, sep=',', encoding='utf-8')

tourism_restaurants_locations.to_csv("tourism_restaurants_locations.csv", index=False, na_rep='N/A', header=True, sep=',', encoding='utf-8')

##### 2.3 TripAdvisor API Data <br>
https://tripadvisor-content-api.readme.io/reference/getlocationdetails

In [12]:
import os
from hook.tripadvisor_data import TripAdvisorApi

# Ensure the API key is set
api_key = os.getenv('TRIPADVISOR_API_KEY')
print(f"API KEY: {api_key}")

# Initialize the API client
thai_tourism_api = TripAdvisorApi(api_key)
print(thai_tourism_api)

# Test API request
response = thai_tourism_api.test_api_request()

API KEY: 65DC791943D1403C8837357A129B28EB


TypeError: TripAdvisorApi.__init__() missing 1 required positional argument: 'language'

In [ ]:
# Import necessary libraries
import geopandas as gpd
import matplotlib.pyplot as plt

# Load the GeoJSON file
geojson_file = 'hook/data/thai_province_coordinates_geo.json'
gdf = gpd.read_file(geojson_file)

# Check for empty geometries and remove them
gdf = gdf[~gdf.is_empty]

# Ensure the CRS is set to WGS 84 (EPSG:4326)
if gdf.crs is None:
    gdf.set_crs(epsg=4326, inplace=True)

# Initialize a Matplotlib figure and axis
fig, ax = plt.subplots(figsize=(12, 8))

# Plot all provinces in light blue
gdf.plot(ax=ax, color='lightblue', edgecolor='black')

# Highlight Bangkok in red, checking if it exists in the data
bangkok = gdf[gdf['name'] == 'Bangkok Metropolis']
if not bangkok.empty:
    bangkok.plot(ax=ax, color='red', edgecolor='black')

# Set the aspect of the plot to equal to ensure accurate representation
ax.set_aspect('equal')

# Add title and labels
ax.set_title('Map of Thai Provinces with Bangkok Highlighted', fontsize=15)
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)

# Show the plot
plt.show()
